In [1]:
import os
import glob
import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.preprocessing import MinMaxScaler
import pingouin as pg
import numpy as np

In [2]:
homedir = '/home/raghuram/Desktop/radiomics/TEXTURES/'
os.chdir(homedir)

In [34]:
t1ce_file = 'expt_t1ce.csv'
t1w_file = 'expt_t1w.csv'
t2f_file = 'expt_t2f.csv'
t2w_file = 'expt_t2w.csv'

In [60]:
def pre_process_dataframe(csv_file_name, experiment_number):
    # Binarize magnetization strength values
    # Scanner names and manufacturer are categorical 
    # One hot encode them
    
    # Code for the above here
    # Experiments in ascending order
    
    experiment_df = pd.read_csv(csv_file_name)
    experiment_df = experiment_df[experiment_df['experiment_number'] == experiment_number]
    fill_values = {'mag_field_strength':1.5}
    experiment_df.fillna(value=fill_values, inplace=True)
    experiment_df['mag_field_strength_binarized'] = (experiment_df['mag_field_strength']>=1.5).astype(int)
    experiment_df.drop(columns=[ 'Tumor','experiment_number', 'mat_file_name', 'scale', 'algo', 'ng', 'sequence_name', 'flip_angle', 
                                'VOLUME_ET', 'VOLUME_NET', 'VOLUME_ED', 'VOLUME_TC',
                               'VOLUME_BRAIN', 'mag_field_strength'], inplace=True)
    
    print(experiment_df.columns)
    experiment_df.dropna(subset=['VOLUME_WT'], inplace=True)
    experiment_df = pd.get_dummies(experiment_df, columns=['scanner_manufacturer', 'scanner_model'], drop_first=True)
    radiomics_features  = list(experiment_df.columns)[:42]

    radiomics_df = experiment_df[radiomics_features]
    scaled_df = experiment_df.drop(columns=radiomics_features)
    scaled_df = scaled_df - scaled_df.min()/(scaled_df.max()-scaled_df.min())
  
    
    return (scaled_df, radiomics_df)
        


In [61]:
def results_summary_to_dataframe(results,response_variable):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    ci_bounds = results.conf_int()
 
    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "response_variable":response_variable,
                              })
    #Reordering...
    results_df = results_df[["coeff","pvals", "response_variable"]]
    return results_df

In [62]:
def linear_regression(results_folder, radiomic_df, scaled_df, experiment_number):
    # Load the data corresponding to the sequence argument here
    # Perform regression with Tr and Te 
    
    radiomic_features = list(radiomic_df.columns)
    y = np.array(radiomic_df)
    X = np.array(scaled_df)
    X = sm.add_constant(X)
    result_list = []
    for idx, column in enumerate(radiomic_features):
        
        print('Feature is {}'.format(column))
        model = sm.OLS(y[:, idx], X)
        results = model.fit()
        results_df = results_summary_to_dataframe(results, column)
        result_list.append(results_df)
       
        
    pd.concat(result_list).to_csv(os.path.join(results_folder, str(experiment_number)+'_'+'t1w.csv'), index=False)

In [63]:
results_folder = '/home/raghuram/Desktop/radiomics/TEXTURES/results/t1ce/linear_regression'
for expt_number in range(1, 26):
    scaled_df, radiomics_df = pre_process_dataframe(t1ce_file, expt_number)
    linear_regression(results_folder, radiomics_df, scaled_df, expt_number) 

Index(['GLCM_Contrast', 'GLCM_Correlation', 'GLCM_Dissimilarity',
       'GLCM_Energy', 'GLCM_Entropy', 'GLCM_Homogeneity', 'GLCM_SumAverage',
       'GLCM_Variance', 'GLRLM_GLN', 'GLRLM_GLV', 'GLRLM_HGRE', 'GLRLM_LGRE',
       'GLRLM_LRE', 'GLRLM_LRHGE', 'GLRLM_LRLGE', 'GLRLM_RLN', 'GLRLM_RLV',
       'GLRLM_RP', 'GLRLM_SRE', 'GLRLM_SRHGE', 'GLRLM_SRLGE', 'GLSZM_GLN',
       'GLSZM_GLV', 'GLSZM_HGZE', 'GLSZM_LGZE', 'GLSZM_LZE', 'GLSZM_LZHGE',
       'GLSZM_LZLGE', 'GLSZM_SZE', 'GLSZM_SZHGE', 'GLSZM_SZLGE', 'GLSZM_ZP',
       'GLSZM_ZSN', 'GLSZM_ZSV', 'Global_Kurtosis', 'Global_Skewness',
       'Global_Variance', 'NGTDM_Busyness', 'NGTDM_Coarseness',
       'NGTDM_Complexity', 'NGTDM_Contrast', 'NGTDM_Strength', 'scanner_model',
       'scanner_manufacturer', 'repetition_time', 'excitation_time',
       'VOLUME_WT', 'mag_field_strength_binarized'],
      dtype='object')
Feature is GLCM_Contrast
Feature is GLCM_Correlation
Feature is GLCM_Dissimilarity
Feature is GLCM_Energy
Feature is

Feature is GLRLM_LRLGE
Feature is GLRLM_RLN
Feature is GLRLM_RLV
Feature is GLRLM_RP
Feature is GLRLM_SRE
Feature is GLRLM_SRHGE
Feature is GLRLM_SRLGE
Feature is GLSZM_GLN
Feature is GLSZM_GLV
Feature is GLSZM_HGZE
Feature is GLSZM_LGZE
Feature is GLSZM_LZE
Feature is GLSZM_LZHGE
Feature is GLSZM_LZLGE
Feature is GLSZM_SZE
Feature is GLSZM_SZHGE
Feature is GLSZM_SZLGE
Feature is GLSZM_ZP
Feature is GLSZM_ZSN
Feature is GLSZM_ZSV
Feature is Global_Kurtosis
Feature is Global_Skewness
Feature is Global_Variance
Feature is NGTDM_Busyness
Feature is NGTDM_Coarseness
Feature is NGTDM_Complexity
Feature is NGTDM_Contrast
Feature is NGTDM_Strength
Index(['GLCM_Contrast', 'GLCM_Correlation', 'GLCM_Dissimilarity',
       'GLCM_Energy', 'GLCM_Entropy', 'GLCM_Homogeneity', 'GLCM_SumAverage',
       'GLCM_Variance', 'GLRLM_GLN', 'GLRLM_GLV', 'GLRLM_HGRE', 'GLRLM_LGRE',
       'GLRLM_LRE', 'GLRLM_LRHGE', 'GLRLM_LRLGE', 'GLRLM_RLN', 'GLRLM_RLV',
       'GLRLM_RP', 'GLRLM_SRE', 'GLRLM_SRHGE', 'GLRLM_

Feature is GLRLM_SRE
Feature is GLRLM_SRHGE
Feature is GLRLM_SRLGE
Feature is GLSZM_GLN
Feature is GLSZM_GLV
Feature is GLSZM_HGZE
Feature is GLSZM_LGZE
Feature is GLSZM_LZE
Feature is GLSZM_LZHGE
Feature is GLSZM_LZLGE
Feature is GLSZM_SZE
Feature is GLSZM_SZHGE
Feature is GLSZM_SZLGE
Feature is GLSZM_ZP
Feature is GLSZM_ZSN
Feature is GLSZM_ZSV
Feature is Global_Kurtosis
Feature is Global_Skewness
Feature is Global_Variance
Feature is NGTDM_Busyness
Feature is NGTDM_Coarseness
Feature is NGTDM_Complexity
Feature is NGTDM_Contrast
Feature is NGTDM_Strength
Index(['GLCM_Contrast', 'GLCM_Correlation', 'GLCM_Dissimilarity',
       'GLCM_Energy', 'GLCM_Entropy', 'GLCM_Homogeneity', 'GLCM_SumAverage',
       'GLCM_Variance', 'GLRLM_GLN', 'GLRLM_GLV', 'GLRLM_HGRE', 'GLRLM_LGRE',
       'GLRLM_LRE', 'GLRLM_LRHGE', 'GLRLM_LRLGE', 'GLRLM_RLN', 'GLRLM_RLV',
       'GLRLM_RP', 'GLRLM_SRE', 'GLRLM_SRHGE', 'GLRLM_SRLGE', 'GLSZM_GLN',
       'GLSZM_GLV', 'GLSZM_HGZE', 'GLSZM_LGZE', 'GLSZM_LZE', 'GL

Feature is GLRLM_SRLGE
Feature is GLSZM_GLN
Feature is GLSZM_GLV
Feature is GLSZM_HGZE
Feature is GLSZM_LGZE
Feature is GLSZM_LZE
Feature is GLSZM_LZHGE
Feature is GLSZM_LZLGE
Feature is GLSZM_SZE
Feature is GLSZM_SZHGE
Feature is GLSZM_SZLGE
Feature is GLSZM_ZP
Feature is GLSZM_ZSN
Feature is GLSZM_ZSV
Feature is Global_Kurtosis
Feature is Global_Skewness
Feature is Global_Variance
Feature is NGTDM_Busyness
Feature is NGTDM_Coarseness
Feature is NGTDM_Complexity
Feature is NGTDM_Contrast
Feature is NGTDM_Strength
Index(['GLCM_Contrast', 'GLCM_Correlation', 'GLCM_Dissimilarity',
       'GLCM_Energy', 'GLCM_Entropy', 'GLCM_Homogeneity', 'GLCM_SumAverage',
       'GLCM_Variance', 'GLRLM_GLN', 'GLRLM_GLV', 'GLRLM_HGRE', 'GLRLM_LGRE',
       'GLRLM_LRE', 'GLRLM_LRHGE', 'GLRLM_LRLGE', 'GLRLM_RLN', 'GLRLM_RLV',
       'GLRLM_RP', 'GLRLM_SRE', 'GLRLM_SRHGE', 'GLRLM_SRLGE', 'GLSZM_GLN',
       'GLSZM_GLV', 'GLSZM_HGZE', 'GLSZM_LGZE', 'GLSZM_LZE', 'GLSZM_LZHGE',
       'GLSZM_LZLGE', 'GLSZM_SZE

Feature is GLRLM_SRLGE
Feature is GLSZM_GLN
Feature is GLSZM_GLV
Feature is GLSZM_HGZE
Feature is GLSZM_LGZE
Feature is GLSZM_LZE
Feature is GLSZM_LZHGE
Feature is GLSZM_LZLGE
Feature is GLSZM_SZE
Feature is GLSZM_SZHGE
Feature is GLSZM_SZLGE
Feature is GLSZM_ZP
Feature is GLSZM_ZSN
Feature is GLSZM_ZSV
Feature is Global_Kurtosis
Feature is Global_Skewness
Feature is Global_Variance
Feature is NGTDM_Busyness
Feature is NGTDM_Coarseness
Feature is NGTDM_Complexity
Feature is NGTDM_Contrast
Feature is NGTDM_Strength
Index(['GLCM_Contrast', 'GLCM_Correlation', 'GLCM_Dissimilarity',
       'GLCM_Energy', 'GLCM_Entropy', 'GLCM_Homogeneity', 'GLCM_SumAverage',
       'GLCM_Variance', 'GLRLM_GLN', 'GLRLM_GLV', 'GLRLM_HGRE', 'GLRLM_LGRE',
       'GLRLM_LRE', 'GLRLM_LRHGE', 'GLRLM_LRLGE', 'GLRLM_RLN', 'GLRLM_RLV',
       'GLRLM_RP', 'GLRLM_SRE', 'GLRLM_SRHGE', 'GLRLM_SRLGE', 'GLSZM_GLN',
       'GLSZM_GLV', 'GLSZM_HGZE', 'GLSZM_LGZE', 'GLSZM_LZE', 'GLSZM_LZHGE',
       'GLSZM_LZLGE', 'GLSZM_SZE

Feature is GLRLM_LRLGE
Feature is GLRLM_RLN
Feature is GLRLM_RLV
Feature is GLRLM_RP
Feature is GLRLM_SRE
Feature is GLRLM_SRHGE
Feature is GLRLM_SRLGE
Feature is GLSZM_GLN
Feature is GLSZM_GLV
Feature is GLSZM_HGZE
Feature is GLSZM_LGZE
Feature is GLSZM_LZE
Feature is GLSZM_LZHGE
Feature is GLSZM_LZLGE
Feature is GLSZM_SZE
Feature is GLSZM_SZHGE
Feature is GLSZM_SZLGE
Feature is GLSZM_ZP
Feature is GLSZM_ZSN
Feature is GLSZM_ZSV
Feature is Global_Kurtosis
Feature is Global_Skewness
Feature is Global_Variance
Feature is NGTDM_Busyness
Feature is NGTDM_Coarseness
Feature is NGTDM_Complexity
Feature is NGTDM_Contrast
Feature is NGTDM_Strength


/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/raghuram/Desktop/radiomics_venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [53]:
df = pd.read_csv(t1ce_file)
clinical_df = pd.read_csv('TCGA_LGG_radiomicFeatures_combined.csv')
clinical_df = clinical_df[['ID', 'VOLUME_WT']]

In [54]:
df['ID'] = df['Tumor']
df = pd.merge(df, clinical_df, how='inner', on='ID')
df.drop(columns=['ID'], inplace=True)
df.to_csv('expt_t1ce.csv', index=False)